In [1]:
import os
import os.path
from glob import glob
import rioxarray
from dask.distributed import Client, LocalCluster, Lock
import xarray as xr

In [2]:
# set up the dask client
# docs.das.org/en/latest/deploying-python.html
cluster = LocalCluster()
# client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB')
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 3
Total threads: 6,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52284,Workers: 3
Dashboard: http://127.0.0.1:8787/status,Total threads: 6
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:52300,Total threads: 2
Dashboard: http://127.0.0.1:52305/status,Memory: 10.67 GiB
Nanny: tcp://127.0.0.1:52287,


In [3]:
# # grab the rasters we need.
swir_2018 = sorted(glob(r'W:\Data\WaterMask\VIIRS\SWIR\2018\*_SWIR.tif'))
green_2018 = sorted(glob(r'W:\Data\WaterMask\VIIRS\GREEN\2018\*_GREEN.tif'))

output_root = r'W:\Data\WaterMask\VIIRS\GreenSWIR_NDWI\2018'
print(f'len swir {len(swir_2018)} len green {len(green_2018)}')

len swir 46 len green 46


In [4]:
for s, g in zip(swir_2018, green_2018):
    swir = rioxarray.open_rasterio(s, masked=True, chunks=(4500, 4500)).squeeze('band', drop=True)
    green = rioxarray.open_rasterio(g, masked=True, chunks=(4500, 4500)).squeeze('band', drop=True)

    # get the data from the filename
    # filename example: 2018001.1_km_8_days_QA.tif
    filename = os.path.split(s)[-1]
    name_chunks = filename.split('.')[0:3]
    unique_id = '.'.join(name_chunks)

    with xr.set_options(keep_attrs=True):
        # ndwi
        ndwi = (green - swir) / (green + swir)

    ndwi.rio.to_raster(os.path.join(output_root, f'{unique_id}_NDWI.tif'), tiled=True, lock=Lock('rio', client=client))